In [21]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


In [22]:
train__set_features = pd.read_csv("training_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_h1n1

y_seasonal = train__set_labels['seasonal_vaccine']

train__set_features.shape

(26707, 36)

In [23]:
y_seasonal.shape

(26707,)

In [24]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)
train.shape



(26707, 38)

On va gerer les valeurs manquantes et les valeurs categoriels 

In [25]:
train.select_dtypes(include='object').isna().sum()
# train = train.drop(['employment_industry'], axis=1)
# la moitié de employement _industry et _occupation est manquante 


age_group                    0
education                 1407
race                         0
sex                          0
income_poverty            4423
marital_status            1408
rent_or_own               2042
employment_status         1463
hhs_geo_region               0
census_msa                   0
employment_industry      13330
employment_occupation    13470
dtype: int64

In [26]:
train = train.drop(["employment_industry", "employment_occupation"], axis=1)

train = pd.get_dummies(train, columns=['education', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status'])

In [27]:
from sklearn.neighbors import KNeighborsRegressor
def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    
    knn.fit(X_train, y_train)
    print(X_na)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [28]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.1

nulls = train[cond.index[cond]].isna().sum() > 0

for col in nulls[nulls == True].index:
    train = manage_na(train, col)




C:\Users\Amélie\AppData\Local\Temp\ipykernel_23752\397326579.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.1


       respondent_id  h1n1_vaccine  seasonal_vaccine  education_12 Years  \
64                64             0                 0                   0   
238              238             0                 0                   0   
730              730             0                 1                   0   
1172            1172             0                 1                   1   
4914            4914             0                 0                   1   
4939            4939             1                 0                   0   
6614            6614             0                 0                   0   
6926            6926             0                 0                   0   
6942            6942             0                 0                   1   
7464            7464             0                 0                   0   
7884            7884             0                 1                   0   
10364          10364             0                 0                   0   
10879       

In [36]:
nulls = train.isna().sum().sort_values() > 0


for col in nulls[nulls == True].index:
    train = manage_na(train, col)

train.isna().sum()



       respondent_id  h1n1_concern  h1n1_knowledge  behavioral_wash_hands  \
1231            1231           1.0             0.0                    1.0   
2562            2562           1.0             1.0                    0.0   
4092            4092           3.0             0.0                    0.0   
4664            4664           1.0             1.0                    1.0   
5796            5796           3.0             0.0                    1.0   
6004            6004           3.0             2.0                    1.0   
6064            6064           2.6             2.0                    1.0   
10081          10081           3.0             2.0                    1.0   
11032          11032           1.2             0.0                    1.0   
13909          13909           0.0             2.0                    1.0   
15277          15277           1.6             1.4                    1.0   
18265          18265           2.0             1.0                    1.0   

respondent_id                               0
h1n1_concern                                0
h1n1_knowledge                              0
behavioral_antiviral_meds                   0
behavioral_avoidance                        0
behavioral_face_mask                        0
behavioral_wash_hands                       0
behavioral_large_gatherings                 0
behavioral_outside_home                     0
behavioral_touch_face                       0
doctor_recc_h1n1                            0
doctor_recc_seasonal                        0
chronic_med_condition                       0
child_under_6_months                        0
health_worker                               0
health_insurance                            0
opinion_h1n1_vacc_effective                 0
opinion_h1n1_risk                           0
opinion_h1n1_sick_from_vacc                 0
opinion_seas_vacc_effective                 0
opinion_seas_risk                           0
opinion_seas_sick_from_vacc       

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

X = train.drop('seasonal_vaccine')
y = train["seasonal_vaccine"]
X_train, X_test, y_train, y_test = train_test_split(X, y, suffle=True, test_size=0.2, random_state=32,)

KeyError: "['seasonal_vaccine'] not found in axis"